In [54]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import torch
from transformers import BertForSequenceClassification

from tqdm.notebook import tqdm

In [55]:
class_label = {0:'hate_speech',1:'offensive_language',2:'neither'}
class_label

{0: 'hate_speech', 1: 'offensive_language', 2: 'neither'}

In [56]:
test_data = pd.read_excel('data/Hate-Speech-CTS.xlsx', engine="openpyxl")
test_data.head()

,class,tweet
0,NaN,"RT @RedTvvitter: ""Does my Mewtwo make your pus..."
1,NaN,@kieffer_jason and who said I didn't like myse...
2,NaN,Get this bitch off my TV bruh
3,NaN,Lmfao bitch got her shit rocked
4,NaN,Verizon customer service is trash as fuck. I'v...


In [57]:
#test_data = test_data[:10].copy()

In [58]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(class_label),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [59]:
model_name = "model/hatespeech-finetuned-bert-20210724120229"

model.load_state_dict(torch.load(model_name, map_location=torch.device('cpu')))

<All keys matched successfully>

In [60]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [61]:
preds = []

with torch.no_grad():
    for i, data in enumerate(test_data['tweet']):
        input = tokenizer(data, return_tensors='pt')
        pred = model(**input)
        preds.append(pred['logits'].argmax().detach().numpy())
        

In [62]:
result = pd.DataFrame({
    'class':preds,
    'tweet':test_data['tweet']
})

result.to_csv('data/result.csv', index=False)